In [ ]:
pip install mlflow scikit-learn


In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)


In [ ]:
# Set the MLflow experiment name
mlflow.set_experiment("Iris Random Forest Experiment")


In [ ]:
with mlflow.start_run(run_name="RandomForest_Experiment") as run:
    # Set model parameters
    params = {"n_estimators": 100, "max_depth": 5, "min_samples_split": 2}
    
    # Train the Random Forest model
    model = RandomForestClassifier(
        n_estimators=params["n_estimators"], 
        max_depth=params["max_depth"], 
        min_samples_split=params["min_samples_split"],
        random_state=42
    )
    model.fit(X_train, y_train)

    # Make predictions and calculate accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Log parameters, metrics, and the model
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(model, "random_forest_model")
    
    print(f"Model logged with accuracy: {accuracy:.4f}")


In [ ]:
mlflow ui


In [ ]:
# Register the model
model_uri = "runs:/{}/random_forest_model".format(run.info.run_id)
mlflow.register_model(model_uri=model_uri, name="Iris_RandomForest_Model")
print("Model registered successfully!")


In [ ]:
name: iris_random_forest

entry_points:
  main:
    parameters:
      n_estimators: {type: int, default: 100}
      max_depth: {type: int, default: 5}
      min_samples_split: {type: int, default: 2}
    command: >
      python train.py --n_estimators {n_estimators}
                      --max_depth {max_depth}
                      --min_samples_split {min_samples_split}


In [ ]:
import argparse
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument("--n_estimators", type=int, default=100)
parser.add_argument("--max_depth", type=int, default=5)
parser.add_argument("--min_samples_split", type=int, default=2)
args = parser.parse_args()

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run():
    model = RandomForestClassifier(
        n_estimators=args.n_estimators,
        max_depth=args.max_depth,
        min_samples_split=args.min_samples_split,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_params(vars(args))
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(model, "model")


In [ ]:
mlflow run . -P n_estimators=150 -P max_depth=10


In [ ]:
mlflow models serve -m models:/Iris_RandomForest_Model/1 --port 1234


In [ ]:
curl -X POST -H "Content-Type:application/json; format=pandas-split" \
     --data '{"columns":["feature1", "feature2", "feature3", "feature4"],"data":[[5.1,3.5,1.4,0.2]]}' \
     http://127.0.0.1:1234/invocations
